<a href="https://colab.research.google.com/github/ginny0410/aop113b/blob/main/EX05_02_Ask_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.1 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN' ) )
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import ( Configuration, ApiClient, MessagingApi,ReplyMessageRequest,TextMessage)
app = Flask(__name__)


configuration = Configuration(access_token=userdata.get('LINE_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANELSECRET'))

@app.route("/callback", methods=['POST'])
def callback():
  signature = request.headers ['X-Line-Signature' ]
  body = request. get_data(as_text=True)
  try:
    handler. handle(body, signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

import google.generativeai as genai
genai.configure(api_key=userdata.get('Google_Api_Key'))
model = genai. GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler. add(MessageEvent, message=TextMessageContent)
def handle_message(event):
  with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    response = ask_gemini(event.message.text)
    line_bot_api.reply_message(
      ReplyMessageRequest(
          reply_token=event.reply_token,
          messages=[
              TextMessage(text=response)
            ]
          )
      )

ngrok_start() # for colab
if __name__ == "__main__":
  app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f8bd-35-185-21-10.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
